In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logEx').getOrCreate()

24/11/01 02:59:07 WARN Utils: Your hostname, AlienEE resolves to a loopback address: 127.0.1.1; using 10.0.0.176 instead (on interface wlp4s0)
24/11/01 02:59:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/01 02:59:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/01 02:59:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/01 02:59:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [ ]:
df = spark.read.csv('datasets/customer_churn.csv', inferSchema=True, header=True)


In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
df.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [12]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [13]:
my_final_cols = df.select([
 'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Churn'])

In [15]:
my_final_cols.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Churn: integer (nullable = true)



In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
assembler = VectorAssembler(inputCols=[
 'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites'], outputCol='features')

In [19]:
assembled_data = assembler.transform(df)

In [21]:
final_data = assembled_data.select('features', 'churn')

In [22]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                618|
|   mean|0.16990291262135923|
| stddev| 0.3758516944638849|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [26]:
lr_model = LogisticRegression(labelCol='churn').fit(train_data)

In [27]:
from pyspark.ml.classification import BinaryLogisticRegressionSummary

In [28]:
train_sum = lr_model.summary

In [29]:
train_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                618|                618|
|   mean|0.16990291262135923|0.11165048543689321|
| stddev| 0.3758516944638849| 0.3151910652689442|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [30]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [31]:
pred_and_labels = lr_model.evaluate(test_data)

In [32]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,4.5...|    0|[5.22979568501927...|[0.99467390001750...|       0.0|
|[27.0,8628.8,5.3,...|    0|[4.83655047724761...|[0.99212808205003...|       0.0|
|[28.0,9090.43,5.7...|    0|[1.31742544255929...|[0.78875304834988...|       0.0|
|[28.0,11128.95,5....|    0|[3.67711769705234...|[0.97532830968671...|       0.0|
|[28.0,11204.23,3....|    0|[1.13909139379445...|[0.75751277875533...|       0.0|
|[29.0,12711.15,5....|    0|[4.33574244243402...|[0.98707703905117...|       0.0|
|[29.0,13255.05,4....|    0|[3.64217070659147...|[0.97447326392834...|       0.0|
|[30.0,8874.83,5.5...|    0|[2.42132872900444...|[0.91843933302349...|       0.0|
|[30.0,10960.52,5....|    0|[2.11867549907598...|[0.89270513123661...|       0.0|
|[31.0,5387.75,6

In [34]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [35]:
auc = evaluator.evaluate(pred_and_labels.predictions)

In [36]:
auc

0.8518987341772152

### Predict on New data

In [38]:
fianl_lr_model = LogisticRegression(labelCol='churn').fit(final_data)

In [39]:
new_data = spark.read.csv(df_path+'new_customers.csv',inferSchema=True, header=True)


In [40]:
new_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [41]:
assembled_new_data = assembler.transform(new_data)

In [42]:
assembled_new_data.printSchema()


root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [43]:
final_results = fianl_lr_model.transform(assembled_new_data)

In [44]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,7.7...|[2.41832342720200...|[0.91821392701292...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,9.2...|[-5.9906663692604...|[0.00249575187936...|       

In [46]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [47]:
assembled_new_data.describe().show()

24/11/01 03:26:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         NULL|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                NULL|            NULL|
| stddev|         NULL| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                NULL|            NULL|
|    min|Andrew Mccall|              22.0|            100.0|          